In [2]:
import pandas as pd

# Đọc file Excel
file_path = "data.xlsx"
df = pd.read_excel(file_path)

# Hiển thị dữ liệu
print(df)


   ID         Tên địa điểm      Vị trí    Loại địa điểm  \
0   1         Vịnh Hạ Long  Quảng Ninh       Thắng cảnh   
1   2         Chùa Một Cột      Hà Nội  Di tích lịch sử   
2   3  Phong Nha - Kẻ Bàng  Quảng Bình   Khu nghỉ dưỡng   
3   4               Đà Lạt    Lâm Đồng       Thắng cảnh   

                                  Mô tả ngắn  Đánh giá         Loại hoạt động  \
0     Kỳ quan thiên nhiên nổi tiếng thế giới       4.8    Thăm quan, Chụp ảnh   
1       Ngôi chùa độc đáo có lịch sử lâu đời       4.6     Thăm quan, Văn hóa   
2  Vườn quốc gia với các hang động nổi tiếng       4.9    Khám phá, Thám hiểm   
3      Thành phố ngàn hoa với khí hậu mát mẻ       4.7  Thăm quan, Nghỉ dưỡng   

                        Ảnh                          Từ khóa  
0  https://example.com/img1  "thăm quan", "chụp ảnh", "biển"  
1  https://example.com/img2     "văn hóa", "lịch sử", "chùa"  
2  https://example.com/img3   "khám phá", "hang động", "núi"  
3  https://example.com/img4    "hoa", "khí hậu

In [11]:
# model.py
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Đọc dữ liệu từ file Excel
def load_data():
    file_path = "data.xlsx"
    df = pd.read_excel(file_path)
    return df

# Hàm gợi ý địa điểm
def recommend_destinations(keyword1, keyword2, keyword3):
    # Đọc dữ liệu từ file Excel
    df = load_data()
    
    # Tạo danh sách các từ khóa người dùng nhập
    user_keywords = [keyword1, keyword2, keyword3]
    user_input = " ".join(user_keywords)
    
    # Sử dụng TfidfVectorizer để chuyển từ khóa thành vector
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(pd.concat([df['Từ khóa'], pd.Series([user_input])], ignore_index=True))
    
    # Tính cosine similarity giữa các địa điểm và từ khóa người dùng nhập
    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()
    
    # Lấy 3 địa điểm có độ tương đồng cao nhất
    similar_indices = cosine_sim.argsort()[-3:][::-1]
    
    # Trả về các địa điểm gợi ý
    recommendations = []
    for idx in similar_indices:
        recommendations.append({
            'Tên địa điểm': df['Tên địa điểm'][idx],
            'Vị trí': df['Vị trí'][idx],
            'Từ khóa': df['Từ khóa'][idx]
        })
    
    return recommendations


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os

# Đọc dữ liệu từ file Excel
def load_data():
    file_path = "data.xlsx"
    # Kiểm tra xem file dữ liệu có tồn tại không
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} không tồn tại.")
    df = pd.read_excel(file_path)
    return df

# Lưu trữ các đối tượng tfidf_matrix và tfidf_vectorizer
def save_tfidf_data():
    df = load_data()

    # Tạo danh sách các từ khóa trong dataset
    corpus = df['Từ khóa'].tolist()

    # Tạo TfidfVectorizer và tính toán tfidf_matrix
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # Kiểm tra và tạo thư mục nếu chưa tồn tại
    output_dir = r'N:/RS_ExploreVN/artifacts/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Lưu các đối tượng vào pickle
    with open(os.path.join(output_dir, 'tfidf_matrix.pkl'), 'wb') as f:
        pickle.dump(tfidf_matrix, f)

    with open(os.path.join(output_dir, 'tfidf_vectorizer.pkl'), 'wb') as f:
        pickle.dump(vectorizer, f)

# Gọi hàm để lưu dữ liệu
save_tfidf_data()


In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os

# Đọc dữ liệu từ file Excel
def load_data():
    file_path = "data.xlsx"
    # Kiểm tra xem file dữ liệu có tồn tại không
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} không tồn tại.")
    df = pd.read_excel(file_path)
    return df

# Tải các đối tượng TF-IDF đã lưu từ pickle
def load_tfidf_data():
    try:
        # Đảm bảo sử dụng đúng đường dẫn đầy đủ cho file pickle
        tfidf_matrix_path = 'N:/RS_ExploreVN/artifacts/tfidf_matrix.pkl'
        tfidf_vectorizer_path = 'N:/RS_ExploreVN/artifacts/tfidf_vectorizer.pkl'

        # Kiểm tra nếu các file pickle tồn tại
        if not os.path.exists(tfidf_matrix_path) or not os.path.exists(tfidf_vectorizer_path):
            raise FileNotFoundError("Không tìm thấy file pickle. Vui lòng kiểm tra lại đường dẫn hoặc tạo lại dữ liệu pickle.")

        # Mở file pickle và load dữ liệu
        with open(tfidf_matrix_path, 'rb') as f:
            tfidf_matrix = pickle.load(f)

        with open(tfidf_vectorizer_path, 'rb') as f:
            vectorizer = pickle.load(f)
        
        return tfidf_matrix, vectorizer
    
    except Exception as e:
        raise RuntimeError(f"Đã xảy ra lỗi khi mở file pickle: {e}")

# Hàm gợi ý địa điểm
def recommend_destinations(keyword1, keyword2, keyword3):
    # Đọc dữ liệu từ file Excel
    df = load_data()

    # Tải các đối tượng TF-IDF
    tfidf_matrix, vectorizer = load_tfidf_data()

    # Tạo danh sách các từ khóa người dùng nhập
    user_keywords = [keyword1, keyword2, keyword3]
    user_input = " ".join(user_keywords)
    
    # Sử dụng TfidfVectorizer để chuyển từ khóa người dùng thành vector
    user_input_tfidf = vectorizer.transform([user_input])
    
    # Tính cosine similarity giữa từ khóa người dùng và tất cả các địa điểm
    cosine_sim = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()
    
    # Lấy 3 địa điểm có độ tương đồng cao nhất
    similar_indices = cosine_sim.argsort()[-3:][::-1]
    
    # Trả về các địa điểm gợi ý
    recommendations = []
    for idx in similar_indices:
        recommendations.append({
            'Tên địa điểm': df['Tên địa điểm'][idx],
            'Vị trí': df['Vị trí'][idx],
            'Từ khóa': df['Từ khóa'][idx]
        })
    
    return recommendations

# Test: Gọi hàm gợi ý với 3 từ khóa
keyword1 = "biển"
keyword2 = "chùa"
keyword3 = "hoa"

recommendations = recommend_destinations(keyword1, keyword2, keyword3)

# Hiển thị kết quả gợi ý
for rec in recommendations:
    print(f"- **{rec['Tên địa điểm']}** (Vị trí: {rec['Vị trí']})")
    print(f"  - Từ khóa: {rec['Từ khóa']}")


- **Chùa Một Cột** (Vị trí: Hà Nội)
  - Từ khóa: "văn hóa", "lịch sử", "chùa", 'biển'
- **Đà Lạt** (Vị trí: Lâm Đồng)
  - Từ khóa: "hoa", "khí hậu", "nhiệt đới"
- **Vịnh Hạ Long** (Vị trí: Quảng Ninh)
  - Từ khóa: "thăm quan", "chụp ảnh", "biển"
